# Eight queens puzzle - using Genetic algorithm
**Karthikeyan Ramachandran**

## About
The eight queens puzzle is the problem of placing eight chess queens on an 8×8 chessboard so that no two queens threaten each other; thus, a solution requires that no two queens share the same row, column, or diagonal. 

## Steps involved
1. Initial population is generated with chromosomes containing random genes.
2. Fitness score of the population is calculated.
3. Based on fitness score, parents are selected for mating.
4. Crossover and Mutation are performed on the selected parents, to produce offsprings.
5. Offsprings are added to the population and Fitness score is calculated.
6. Process is stopped if max-fitness score is found, else repeated from Step 3 to 5

## Import libraries

In [1]:
import numpy as np
import pandas as pd

# Setting seed value for reproducibility
np.random.seed(50)

## Define parameters

In [2]:
# Define sizes of initial-population, children and generations 
initial_population_size = 100
num_of_children = 50
num_of_generation = 500

# position to perform Crossover operation on the combination
crossOver=0.5

# Define fitness parameters
num_of_queens = 8
num_of_checks = 7
target_value = num_of_queens * num_of_checks

## Define functions

In [3]:
# Generate initial-population
def generate_init_pop(pop_size):
    initial_population = []
    for i in range(pop_size):
        initial_population.append(np.random.randint(0,8,8))
    return initial_population

In [4]:
# Fitness score-dependent functions

## Get attack positions
def get_atk_position(pos, dist):
    atk_position = np.array([pos-dist, pos, pos+dist], dtype="int32")
    return atk_position

## Get relative position of the queen (to calculate distance)
def get_rel_pos(position_array, idx_val):
    rel_pos = np.empty(8)
    for i in range(len(position_array)):
        if idx_val < i:
            rel_pos[i] = (i - idx_val)
        else:
            rel_pos[i] = (idx_val - i)
            
    return rel_pos

In [5]:
# Fitness score 

## Max fitness score is 8 queens * 7 attacks from other queens = 56
def fitness_score(position_array):
    score = 0
    for i in range(len(position_array)):
        rel_pos = get_rel_pos(position_array, i)
        for j in range(len(position_array)):
            if i == j:
                continue
            atk_position = get_atk_position(position_array[i], rel_pos[j])
            if position_array[j] in atk_position:
                is_attack = True
            else:
                is_attack = False
                score += 1
            # print(position_array[j], atk_position, is_attack, score)
    return score

In [6]:
# generate probabilty based on fitness score
def get_probability(fitness_value, target_value = 56):
    probability = np.empty(len(fitness_value))
    for i in range(len(fitness_value)):
        probability[i] = (np.round(fitness_value[i]/target_value, 5))
    return probability

In [7]:
# sort dataframe based on probability
def prob_sort(population_dataframe):
    population_dataframe = (population_dataframe.sort_values(["probability"], ascending=False).reset_index(drop=True))
    return population_dataframe

In [8]:
# store generated info into a dataframe
def get_dataframe(combination_array):
    population = []
    fitness = []

    for i in range(len(combination_array)):
        combination = combination_array[i]
        population.append(combination)
        fitness.append(fitness_score(combination))

    pop_df = pd.DataFrame({"combination":population, "fitness":fitness})
    pop_df["probability"] = get_probability(pop_df["fitness"])
    pop_df = prob_sort(pop_df)

    return pop_df

In [9]:
# function to perform One Point Crossover
def crossover(parent1, parent2, crossOver=0.5):
    # get Crossover length to slice
    crossOver_len = round(int(len(parent1) * crossOver))
    
    # Slice and perform crossover
    child1 = np.concatenate((parent1[:crossOver_len], parent2[crossOver_len:]), axis=0)
    child2 = np.concatenate((parent2[:crossOver_len], parent1[crossOver_len:]), axis=0)
    
    return child1, child2

In [10]:
# function to perform Random Resetting mutation
def mutation(combination):
    random_queen = np.random.randint(8)
    random_position = int(np.random.randint(0,8,1))
    combination[random_queen] = random_position
    return combination

In [11]:
# generate offspring combination based on performing Crossover and Mutation on selected parents
def get_children(parent_array):

    children = []
    mutated_children = []

    for i in range(0,num_of_children,2):
        p1 = parent_array[i]
        p2 = parent_array[i+1]
    
        (c1, c2) = crossover(p1, p2)
    
        children.append(c1)
        children.append(c2)
    
    for i in range(num_of_children):
        mutated_children.append(mutation(children[i]))
    
    return mutated_children

In [12]:
# Function to generate the right sequenceinitial_population_size
def generate_sequence(initial_population, children, generations, expected_fitness):
    # get initial population fitness

    init_pop_fitness = get_dataframe(initial_population)
    mating_pool = init_pop_fitness[:children]

    print("expected fitness:", expected_fitness, "\n============================\n")

    for i in range(generations):
        # get expected fitness count
        expected_fitness_count = mating_pool[(mating_pool["fitness"]==expected_fitness)]["fitness"].count()
        if expected_fitness_count > 0:
            print("Gen:", (i-1), 
                ", best-combination:", mating_pool["combination"][0], 
                ", fitness:", mating_pool["fitness"][0],
                 ", probability:", mating_pool["probability"][0]
             )
            break
        gen_pop = []
        gen_pop = get_children(mating_pool["combination"])
        gen_pop = np.concatenate((gen_pop, list(mating_pool["combination"])), axis=0)
        mating_pool = get_dataframe(gen_pop)
    
        print("Gen:", i, 
            ", best-combination:", mating_pool["combination"][0], 
            # mating_pool["combination"][0], mating_pool["combination"][1], mating_pool["combination"][2],
            ", fitness:", mating_pool["fitness"][0],
            ", avg-fitness:", np.round(np.mean(mating_pool["fitness"]), 3),
            ", probability:", mating_pool["probability"][0]
            )
    return mating_pool[mating_pool["probability"]==1]["combination"][0]

## Generate inital-population

In [16]:
# Generate initial-population
init_pop = generate_init_pop(initial_population_size)

# display initial-population
init_pop[:5]

[array([4, 0, 7, 3, 1, 0, 4, 0]),
 array([5, 3, 6, 3, 1, 6, 0, 2]),
 array([0, 4, 3, 7, 4, 2, 7, 7]),
 array([6, 7, 3, 6, 1, 4, 0, 3]),
 array([2, 5, 2, 7, 7, 6, 1, 6])]

## Generate right sequence

In [17]:
# store right sequence
right_sequence = generate_sequence(init_pop, num_of_children, num_of_generation, target_value)

expected fitness: 56 

Gen: 0 , best-combination: [4 1 7 5 3 0 2 6] , fitness: 52 , avg-fitness: 43.48 , probability: 0.92857
Gen: 1 , best-combination: [4 1 7 5 3 0 2 6] , fitness: 52 , avg-fitness: 43.427 , probability: 0.92857
Gen: 2 , best-combination: [7 1 3 6 0 7 5 4] , fitness: 52 , avg-fitness: 43.55 , probability: 0.92857
Gen: 3 , best-combination: [7 1 3 6 0 7 5 4] , fitness: 52 , avg-fitness: 43.576 , probability: 0.92857
Gen: 4 , best-combination: [7 1 3 6 0 7 5 4] , fitness: 52 , avg-fitness: 43.613 , probability: 0.92857
Gen: 5 , best-combination: [4 1 7 5 3 0 2 6] , fitness: 52 , avg-fitness: 43.726 , probability: 0.92857
Gen: 6 , best-combination: [7 1 3 6 0 7 5 4] , fitness: 52 , avg-fitness: 43.805 , probability: 0.92857
Gen: 7 , best-combination: [2 6 3 7 4 1 0 2] , fitness: 52 , avg-fitness: 43.933 , probability: 0.92857
Gen: 8 , best-combination: [2 6 3 7 4 1 0 2] , fitness: 52 , avg-fitness: 43.952 , probability: 0.92857
Gen: 9 , best-combination: [2 6 3 7 4 1 0 2

In [18]:
# Display the right sequence
print("Sequence :", right_sequence)

Sequence : [3 6 0 7 4 1 5 2]
